**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

See [Configure Service Account](https://kubernetes.io/docs/tasks/configure-pod-container/configure-service-account/)

In [1]:
kubectl get serviceaccount default -o yaml

apiVersion: v1
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-12-16T14:34:19Z"
  name: default
  namespace: default
  resourceVersion: "405"
  uid: 0b20df1e-9b31-45a4-afb9-4178a0105b86
secrets:
- name: default-token-62zgf


In [2]:
kubectl patch serviceaccount default  --patch "automountServiceAccountToken: false"

serviceaccount/default patched


In [3]:
kubectl get serviceaccount default -o yaml | grep -E '(^|automountServiceAccountToken.*)'

apiVersion: v1
automountServiceAccountToken: false
kind: ServiceAccount
metadata:
  creationTimestamp: "2021-12-16T14:34:19Z"
  name: default
  namespace: default
  resourceVersion: "505"
  uid: 0b20df1e-9b31-45a4-afb9-4178a0105b86
secrets:
- name: default-token-62zgf


# ☠️ **this can be overwritten** ☠️



In [4]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-with-token-again
  labels:
    app: nginx-with-token-again
spec:
  replicas: 1
  selector:
    matchLabels:
      app: nginx-with-token-again
  template:
    metadata:
      labels:
        app: nginx-with-token-again
    spec:
      automountServiceAccountToken: true
      containers:
      - name: nginx
        image: nginxinc/nginx-unprivileged
        ports:
        - containerPort: 8080
EOF

deployment.apps/nginx-with-token-again created


In [7]:
kubectl get pods
POD=$(kubectl get pods -o jsonpath='{.items[0].metadata.name}' )
echo $POD

NAME                                      READY   STATUS    RESTARTS   AGE
nginx-with-token-again-66868cff8b-24zjt   1/1     Running   0          21s
nginx-with-token-again-66868cff8b-24zjt


In [9]:
kubectl exec  $POD -- df

Filesystem                    1K-blocks     Used Available Use% Mounted on
overlay                        61796348 13900232  44734004  24% /
tmpfs                             65536        0     65536   0% /dev
tmpfs                           7888556        0   7888556   0% /sys/fs/cgroup
/dev/mapper/sirius--vg-docker  61796348 13900232  44734004  24% /etc/hosts
shm                               65536        0     65536   0% /dev/shm
tmpfs                          15777116       12  15777104   1% /run/secrets/kubernetes.io/serviceaccount
tmpfs                           7888556        0   7888556   0% /proc/acpi
tmpfs                           7888556        0   7888556   0% /proc/scsi
tmpfs                           7888556        0   7888556   0% /sys/firmware


In [10]:
kubectl get pods  -o yaml | grep -i token

    generateName: nginx-with-token-again-66868cff8b-
      app: nginx-with-token-again
    name: nginx-with-token-again-66868cff8b-24zjt
      name: nginx-with-token-again-66868cff8b
    automountServiceAccountToken: true
        - serviceAccountToken:
            path: token


In [11]:
kubectl get pods -A -o jsonpath='{.items[*].spec.automountServiceAccountToken}'

true